# Title: Changed Values

In [17]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib.colors as colors

from grid2 import *
import torch
import sinkhorn_numpy as sk

In [18]:
%matplotlib qt

In [142]:
def calculate_values(seq, board, epsilon=1e-7):
    d = np.ones(1)
    while np.max(d)>epsilon:
        d = update(board, seq)
        #, targets=[State.to_token(targetA[0],targetA[1]).item(), State.to_token(targetB[0], targetB[1]).item()], 
        #           values={State.to_token(targetA[0],targetA[1]).item():0, State.to_token(targetB[0], targetB[1]).item(): -20})


        
# Configuration
agent = [9,18]
obstacle = np.concatenate([np.arange(5,14).reshape(-1,1), np.ones(14-5).reshape(-1,1)+7], axis=1)

targetA = [2,2]
targetB = [7,2]
beta = 3
shape_row, shape_col = 20, 20

# Initialization

seq, board = set_board(shape_row,shape_col,obstacle,targets=[targetA,], agents=[agent,],beta=beta)
calculate_values(seq, board)

seq2, board2 = set_board(shape_row,shape_col,obstacle,targets=[targetB,], agents=[agent,],beta=beta)
calculate_values(seq2, board2)


value = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq))).reshape(shape_row+2, shape_col+2)
value2 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq2))).reshape(shape_row+2, shape_col+2)

In [143]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

# x,y = State.to_coord(State.targets).reshape(2,-1)
y,x = targetB
ax.plot(x+1,y+1,'x')
y,x = 9,18
ax.plot(x+1,y+1,'o')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')

## Trace the paths: MC

### Original $\eta=0$

In [132]:
eta_raw = 0
volume = 1000
depth = 50
eta = eta_raw / beta

In [133]:
def distribution_to_value(prob_dist, hypo):
    return prob_dist[hypo]-1./len(prob_dist)

In [134]:
# Make a multiprocessing version?
def monte_carlo(boards, eta, hypo=0, depth=50, volume=1000, fvalue=distribution_to_value, init=None):
    """
    fvalue(prob_distribution, true_hypo) : do we need a possible **args?
    """
    if init is None:
        init = np.ones(len(boards))/len(boards)
    else:
        init = np.asarray(init, dtype=float)
    
    data = np.zeros([volume, depth+1, 2], dtype=np.int32)
    for i in range(volume):
        
        data[i,:,:] = single_path(boards, eta, hypo, depth, fvalue, init).copy()

    return data
        
def single_path(boards, eta, hypo, depth, fvalue, init):

    path = np.zeros([depth+1, 2], dtype=np.int32)
    path[0] = State.to_coord(boards[hypo][1,1].agents).reshape(-1) + 1
    current = boards[hypo][tuple(path[0])]
    width = len(boards)
    target = current.targets
    posterior = init.copy()
    
    for step in range(depth):
        if current.token in target:
            path[step + 1] = path[step].copy()
            continue
        length = len(current.Q)
        mat = np.zeros([length, width])
        for i, b in enumerate(boards):
            mat[:, i] = np.array(b[tuple(path[step])].Q).copy()
        
        log_Q = mat[:, hypo].copy()
        mat = np.exp(beta * mat)
        mat = sk.col_normalize(mat, np.ones(width))
        mat = mat * posterior
        # print(type(mat))
        mat = sk.row_normalize(mat, np.ones(length))
        
        for i, row in enumerate(mat):
            log_Q[i] += eta * fvalue(row, hypo)
        
        prob = np.exp(beta * log_Q)
        prob /= np.sum(prob)
        # print(path[step], prob,)
        sample = np.random.choice(length, p=prob)
        posterior = mat[sample,:].copy()
        current = current.action[sample]
        path[step + 1] = current.coord + 1
        
    return path

In [135]:
# generate 1000 episodes
data = monte_carlo([board,board2], eta, hypo=1, volume = volume, depth=depth)

In [136]:
# change data to distributions over time
dist = np.zeros([depth + 1, shape_row+2, shape_col+2])
dist[:, [0, -1], :] = 10 * volume
dist[:, :, [0, -1]] = 10 * volume
for x in obstacle:
    dist[:, int(x[0]), int(x[1])] = 10 * volume

for s in data:
    for i in range(depth + 1):
        # print(i, s[i][1], s[i][0])
        dist[i, s[i][0], s[i][1]] += 1
        
dist /= volume

In [137]:
fig, ax = plt.subplots()
steps = 51
threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
               vmin=0, vmax=1)

y, x = targetB
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = targetB
    ax.plot(x+1,y+1,'x',color='green')
    
    y, x = targetA
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [67]:
ani.save('MC_basic_3_22.gif', writer='imagemagick', fps=2)

MovieWriter imagemagick unavailable. Trying to use pillow instead.


### First Try: $\eta=2$

In [69]:
eta_raw = 2
volume = 1000
depth = 50
eta = eta_raw / beta

In [70]:
# generate 1000 episodes
data = monte_carlo([board,board2], eta, hypo=1, volume = volume, depth=depth)

In [71]:
# change data to distributions over time
dist = np.zeros([depth + 1, shape_row+2, shape_col+2])
dist[:, [0, -1], :] = 10 * volume
dist[:, :, [0, -1]] = 10 * volume
for x in obstacle:
    dist[:, int(x[0]), int(x[1])] = 10 * volume

for s in data:
    for i in range(depth + 1):
        # print(i, s[i][1], s[i][0])
        dist[i, s[i][0], s[i][1]] += 1
        
dist /= volume

# draw animation
fig, ax = plt.subplots()
steps = 51
threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
               vmin=0, vmax=1)

y, x = targetB
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = targetB
    ax.plot(x+1,y+1,'x',color='green')
    
    y, x = targetA
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [72]:
ani.save('MC_trace_path_BI_3_22.gif', writer='imagemagick', fps=2)

MovieWriter imagemagick unavailable. Trying to use pillow instead.


### Second Trial: $\eta=5$

In [74]:
eta_raw = 5
volume = 1000
depth = 50
eta = eta_raw / beta

In [75]:
# generate 1000 episodes
data = monte_carlo([board,board2], eta, hypo=1, volume = volume, depth=depth)
# change data to distributions over time
dist = np.zeros([depth + 1, shape_row+2, shape_col+2])
dist[:, [0, -1], :] = 10 * volume
dist[:, :, [0, -1]] = 10 * volume
for x in obstacle:
    dist[:, int(x[0]), int(x[1])] = 10 * volume

for s in data:
    for i in range(depth + 1):
        # print(i, s[i][1], s[i][0])
        dist[i, s[i][0], s[i][1]] += 1
        
dist /= volume

# draw animation
fig, ax = plt.subplots()
steps = 51
threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
               vmin=0, vmax=1)

y, x = targetB
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = targetB
    ax.plot(x+1,y+1,'x',color='green')
    
    y, x = targetA
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [76]:
ani.save('MC_trace_path_BI_eta5_3_22.gif', writer='imagemagick', fps=2)

MovieWriter imagemagick unavailable. Trying to use pillow instead.


## Positional Value Adjustment

In [151]:
        
# Configuration
agent = [10,18]
obstacle = np.concatenate([np.arange(5,14).reshape(-1,1), np.ones(14-5).reshape(-1,1)+7], axis=1)

targetA = [2,2]
targetB = [7,2]
beta = 3
shape_row, shape_col = 20, 20

# Initialization

seq1, board1 = set_board(shape_row,shape_col,obstacle,targets=[targetA,], agents=[agent,],beta=beta)
calculate_values(seq1, board1)

seq2, board2 = set_board(shape_row,shape_col,obstacle,targets=[targetB,], agents=[agent,],beta=beta)
calculate_values(seq2, board2)


value1 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq1))).reshape(shape_row+2, shape_col+2)
value2 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq2))).reshape(shape_row+2, shape_col+2)

In [152]:
def positional_update(seqs, eta, hypo, fvalue=distribution_to_value):
    """
    This changes the sequences (boards).
    """
    width = len(seqs)
    for i in range(len(seqs[0])):
        if seqs[0][i] is None:
            continue
        mat = np.zeros([len(seqs[0][i].Q),width])
        for j in range(width):
            mat[:, j] = np.array(seqs[j][i].Q).copy()
        
        mat = np.exp(State.beta * mat)
        mat = sk.row_normalize(sk.col_normalize(mat, np.ones(width)), np.ones(mat.shape[0]))
        
        for j in range(width):
            for k in range(mat.shape[0]):
                seqs[j][i].Q[k] += eta * fvalue(mat[k,:], j)
    
    for seq in seqs:
        for node in seq:
            if node is None:
                continue
            print(node.update_V())

In [158]:
eta_raw = 5
eta = eta_raw / State.beta

positional_update([seq1, seq2], eta, 0)

0.023564045101484687
0.009525296917328507
0.007249574742307363
0.00012103099589610977
0.003518811323687032
0.00600393821404932
0.006752684358470873
0.0054379104796815625
0.0036330958225025967
0.0022130189476321505
0.0012884314597059188
0.0007362567468582881
0.00041980649994854957
0.000241198899669115
0.00014023553899633612
8.249878029076285e-05
4.896673009824326e-05
2.9206356867206296e-05
1.752042807723342e-05
7.831729121221542e-06
0.37041197843064855
0.5913292781195425
0.005774853590546858
0.0036690704235984306
0.008425557787081317
0.008876401341623819
0.007623234788565192
0.004376519708696236
0.002091643913460395
0.0009407268237335842
0.0004140764011761888
0.0001814132614672559
7.963515332498616e-05
3.496585091156135e-05
1.5195628042974363e-05
6.395671872994058e-06
2.498366260539342e-06
8.139356069136738e-07
1.4669444681203458e-07
7.053009412061328e-07
0.28452772597997034
0.75805620232158
0
0.3333852363603892
0.3240463025727012
0.31361972231233537
0.3055575938907329
0.289504075373366

In [159]:
threshold=0.0001
fig, ax = plt.subplots()
im = ax.imshow(value.reshape(22,22), cmap="viridis", interpolation='none', vmax=threshold)

# x,y = State.to_coord(State.targets).reshape(2,-1)
y,x = targetB
ax.plot(x+1,y+1,'x')
y,x = 9,18
ax.plot(x+1,y+1,'o')
# plt.axis("off")
cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')

In [160]:
value_ch1 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq1))).reshape(shape_row+2, shape_col+2)
value_ch2 = np.array(list(map(lambda x: x.value if x is not None else 1000002, seq2))).reshape(shape_row+2, shape_col+2)

In [161]:
# 50 step data
steps = 50
single_slice = np.array(list(map(lambda x: 0. if x is not None else 2., seq2))).reshape(22,22)
dist = np.concatenate([single_slice.reshape(1,22,22),]*steps, axis=0)
start = State.to_coord(State.agents[0])
dist[0, start[0], start[1]] = 1.
for i in range(1,steps):
    for a in seq2:
        if a is None:
            continue
        if State.to_token(*a.coord) in a.targets:
            cx, cy = a.coord + 1
            dist[i, cx, cy] += dist[i-1, cx, cy]
            continue
        prob = np.exp(np.array(a.Q)*State.beta)
        prob /= np.sum(prob)
        cx, cy = a.coord + 1
        for k, act in enumerate(a.action):
            x, y = act.coord + 1
            dist[i, x, y] += prob[k] * dist[i-1, cx, cy] 

In [162]:
fig, ax = plt.subplots()

tempA = seq2[50]

threshold=1.5
im = ax.imshow(dist[0], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
               vmin=0, vmax=1)

y, x = State.to_coord(tempA.targets).reshape(2,-1)
ax.plot(x+1,y+1,'x',color='green')
# plt.axis("off")

def animate(i):
    global dist, im
    threshold=1.5
    im = ax.imshow(dist[i], cmap="viridis", interpolation='none', norm=colors.PowerNorm(gamma=0.25),
                   vmin=0., vmax=1.)

    y, x = State.to_coord(tempA.targets).reshape(2,-1)
    ax.plot(x+1,y+1,'x',color='green')
    # plt.axis("off")
    

cbar = fig.colorbar(im, extend='max')
cbar.cmap.set_over('black')
ani = animation.FuncAnimation(fig, animate, frames=steps, interval=500) 
plt.show()

In [163]:
ani.save('Positional_eta5_3_22.gif', writer='imagemagick', fps=2)

MovieWriter imagemagick unavailable. Trying to use pillow instead.
